# ONNX Runtime + YoloV5s Sample Application


# Prerequisites (DO NOT SKIP)

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources and deploy applications on your behalf.

2. This notebook works best in the Test Utility. Please see [here](https://github.com/aws-samples/aws-panorama-samples/blob/main/docs/EnvironmentSetup.md) to set it up 

3. **VERY IMPORTANT** : This example uses a batch of 8, this means we have to use ATLEAST 2 CAMERAS for this example. Batch size 8 is suitable for Jetson Xavier AGX. And for devices using Jetson Xavier NX module, please select batch size 2 instead of 8. please refer to this [link](https://aws.amazon.com/tw/panorama/appliance/) for more information about your device.

4. **PLEASE READ THE [README](README.md) INCLUDE WITH THIS BEFORE YOU START USING THIS NOTEBOOK**

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama', region_name = 'us-west-2')

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def save_cell(line, cell):
    'Save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# application name
app_name = 'onnx_37_app'

## package names and node names
code_package_name = 'onnx_37_app'
camera_node_name = 'abstract_rtsp_media_source'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./onnx_37_app && panorama-cli import-application

## Download Depedencies and Artifacts (One Time Download)

In [ ]:
panorama_test_utility.download_artifacts_gpu_sample('onnx', account_id)

### Build Docker image from Dockerfile

Open a terminal, go to ./ONNX37_opengpu/dependencies/docker and run

```sudo docker build -t  onnx37:latest . ```

### Build app with container

In [ ]:
container_asset_name = 'onnx_37_app'

In [ ]:
%%capture captured_output

# Building container image.This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./onnx_37_app && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

### Special flags in package.json

* Step 1 : Before you deploy the application, open ONNX37_opengpu/onnx_37_app/packages/(account-id)-onnx_37_app-1.0/package.json
* Step 2 : Add the following flags to the package.json

```
"requirements": 
            [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
            }]
```

The assets should look something like this

```
"assets": [
    {
        "name": "onnx_37_app",
        "implementations": [
            {
                "type": "container",
                "assetUri": "9a49a98784f4571adacc417f00942dac7ef2e34686eef21dca9fcb7f4b7ffd70.tar.gz",
                "descriptorUri": "4bab130ec48eea84e072d9fe813b947e9d9610b2924099036b0165026a91d306.json",
                "requirements": 
                [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
                }]
            }
        ]
    }
],
```

### Upload application to Panorama for deploying to devices

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./onnx_37_app && pwd && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

* Step 1: Copy the contents of./graphs/onnx_37_app/graph.json
* Step 2 : Go to the AWS Panorama Console, click Deploy Application
* Step 3 : Paste the contents you just copied
* Step 4 : Select the device
* Step 5: Select atleast 2 cameras
* Step 7: Configure the batch size with desired batch size. 8 for Jetson Xavier AGX and 2 for Jetson NX. For more info about your Panorama spec, please refer to this [link](https://aws.amazon.com/tw/panorama/appliance/)
* Step 8 : Deploy

The deployment should take about 20 minutes